# Libraries

In [ ]:
# Internal
from collections import namedtuple
from dataclasses import dataclass, field
import os
import pickle

# Data
import pandas as pd
import numpy as np
import dill # pickle but for more complex objects (for saving the stats_dict which contains the custom OLS_Stats namedtuple)

# Stats
from scipy.stats import norm, kendalltau, theilslopes, zscore, f_oneway, levene
import statsmodels.api as sm

# Graphing
import matplotlib.pyplot as plt

Classes

In [ ]:
coordinates = namedtuple("coordinates", ["lat", "lon"])

@dataclass
class Site_Data:
    elevation: int
    coords: coordinates = None
    sat_day: pd.DataFrame = field(default_factory = pd.DataFrame)
    sat_month: pd.DataFrame = field(default_factory = pd.DataFrame)
    sat_season: pd.DataFrame = field(default_factory = pd.DataFrame)
    sat_annual: pd.DataFrame = field(default_factory = pd.DataFrame)
    cli_day: pd.DataFrame = field(default_factory = pd.DataFrame)
    cli_month: pd.DataFrame = field(default_factory = pd.DataFrame)
    cli_season: pd.DataFrame = field(default_factory = pd.DataFrame)
    cli_annual: pd.DataFrame = field(default_factory = pd.DataFrame)

@dataclass
class Atmos_Data:
    day: pd.DataFrame = field(default_factory = pd.DataFrame)
    week: pd.DataFrame = field(default_factory = pd.DataFrame)
    month: pd.DataFrame = field(default_factory = pd.DataFrame)
    season: pd.DataFrame = field(default_factory = pd.DataFrame)
    annual: pd.DataFrame = field(default_factory = pd.DataFrame)

# Import

Import

In [ ]:
sites = {}
model_date = "2025-07-10"
site_mapping_coords = {
    "se_st1": coordinates(68.35415, 19.050333),
    "se_sto": coordinates(68.35594288, 19.04520892),
    "fi_ken": coordinates(67.98721, 24.24301),
    "fi_sod": coordinates(67.36239, 26.63859),
    "fi_var": coordinates(67.7549, 29.61),
}
site_mapping_elevation = {
    "se_st1": 351,
    "se_sto": 351,
    "fi_ken": 347,
    "fi_sod": 187,
    "fi_var": 395,
}

for file in os.listdir("../arctic-ecosystems/gppe-model-predictions/"):
    site = file[0:6]
    date = file[22:32]
    
    if date == model_date:
        print(site, date)
        
        coords = site_mapping_coords.get(site)
        elevation = site_mapping_elevation.get(site)
        
        sites[site] = Site_Data(
            sat_day = pd.read_csv(f"../arctic-ecosystems/gppe-model-predictions/{site}_satellite-gppe_{date}.csv"),
            cli_day = pd.read_csv(f"../data/processed/{site}_climate.csv"),
            coords = coords,
            elevation = elevation
            
        )

atmos = Atmos_Data(
    day = pd.read_csv("../data/processed/atmos.csv")
)

Cleanup

In [ ]:
# Reorganise to datetime indices
for key, item in sites.items():
    item.cli_day["date"] = pd.to_datetime(item.cli_day["Unnamed: 0"]).dt.date
    item.sat_day["date"] = pd.to_datetime(item.sat_day["Unnamed: 0"]).dt.date
    
    item.cli_day.set_index("date", inplace = True)
    item.sat_day.set_index("date", inplace = True)
    
    item.cli_day.rename_axis(None, inplace = True)
    item.sat_day.rename_axis(None, inplace = True)
    
    item.sat_day["datetime"] = pd.to_datetime(item.sat_day["Unnamed: 0"])
    item.sat_day.drop("Unnamed: 0", inplace = True, axis = 1)
    item.cli_day.drop("Unnamed: 0", inplace = True, axis = 1)
    
    item.cli_day.index = pd.to_datetime(item.cli_day.index)
    item.sat_day.index = pd.to_datetime(item.sat_day.index)

atmos.day["date"] = pd.to_datetime(atmos.day["Unnamed: 0"]).dt.date
atmos.day.set_index("date", inplace = True)
atmos.day.rename_axis(None, inplace = True)
atmos.day.drop("Unnamed: 0", inplace = True, axis = 1)
atmos.day.index = pd.to_datetime(atmos.day.index)


# Extract satellite scene platforms into new, unfiltered dict per site or plotting data density later on.
density = {}
for key, item in sites.items():
    density[key] = item.sat_day[["platform_id"]].copy()

for key, item in sites.items():
    
    # Remove Pre 2000 values and post 2025 values from satellite data
    item.sat_day = item.sat_day[item.sat_day.index > pd.Timestamp("2000-01-01")]
    item.sat_day = item.sat_day[item.sat_day.index < pd.Timestamp("2025-01-01")]

    # Take only cols I want
    item.sat_day = item.sat_day[["gppe", "ndvi", "gpp", "platform_id", "system:index"]]
    
    # Convert gppe from co2 to C
    item.sat_day["gppe"] = item.sat_day["gppe"] * 0.273
    item.sat_day["gpp"] = item.sat_day["gpp"] * 0.273
    

# Remove incorrect kenttarova value
sites["fi_ken"].cli_day = sites["fi_ken"].cli_day[sites["fi_ken"].cli_day.index > pd.Timestamp("2003-01-01")]

# Time Aggregation 

Sub Functions

In [ ]:
# Aggregation Functions
agg_funcs = {
    "mean": "mean",
    "min": "min",
    # "median": "median",
    "max": "max",
    "q05": lambda x: x.quantile(0.05),
    "q95": lambda x: x.quantile(0.95),
    "std": "std"
}

# AUC
def calc_auc(group, n_bootstrap = 1000, random_seed = 2):
    
    group = group.dropna()
    if len(group) < 3:
        return np.nan, np.nan
    
    days_since_start = (group.index - group.index.min()) / np.timedelta64(1, 'D')
    days_since_start = days_since_start.values
    
    rng = np.random.default_rng(random_seed)
    bootstrapped_aucs = []
    valid_indices = np.arange(len(group))
    for _ in range(n_bootstrap):
        sample_indices = rng.choice(valid_indices, size=len(valid_indices), replace=True)
        sample_x = days_since_start[sample_indices]
        sample_y = group.values[sample_indices]
        sorted_order = np.argsort(sample_x)
        sample_x_sorted = sample_x[sorted_order]
        sample_y_sorted = sample_y[sorted_order]
        auc_sample = np.trapezoid(y=sample_y_sorted, x=sample_x_sorted)
        bootstrapped_aucs.append(auc_sample)

    bootstrapped_aucs = np.array(bootstrapped_aucs)
    return bootstrapped_aucs.mean(), bootstrapped_aucs.std()

# End-of-ablation-season
def find_end_of_ablation_season(group, variable):
    weekly_means = group[variable].resample("W").mean()
    zero_weeks = weekly_means[weekly_means == 0]
    
    if not zero_weeks.empty:
        return zero_weeks.index[0].day_of_year
    else:
        return None
    
# Warmth Index
def find_warmth_index(t, min_months, period_mean = "MS"):
    
    t_mean = t.resample(period_mean).mean()
    
    if t_mean.count() < min_months:
        return np.nan
    else:
        t0 = t_mean[t_mean > 0]
        wi = t0.sum()
        return wi

# Growing Degree Days
def find_gdd(t):
    # https://www.nature.com/articles/s41597-023-01959-w
    
    t = t[t > 5] # all temps above 5
    if t.empty:
        return np.nan
    
    gdd = t.sum()
    
    return gdd

# Snow Gain and Loss
def find_snow_gain(s):
    diff = s.diff()
    diff_filter = diff[diff > 0]
    return diff_filter.sum()
def find_snow_loss(s):
    diff = s.diff()
    diff_filter = diff[diff < 0]
    return diff_filter.sum()

Function

In [ ]:
def calc_aggregates(df, resample_length, is_climate = False, is_satellite = False):
    
    df = df.copy().select_dtypes(include = np.number)
    
    # Make appropriate resampled objects
    if resample_length == "week":
        df_resampled = df.resample("W")
    elif resample_length == "month":
        df_resampled = df.resample("MS")
    elif resample_length == "season":
        df_resampled = df.resample("QS-DEC") # pandas is setup for fiscal quarters starting on jan. We need meteorological seasons/quarters starting on Dec.
    elif resample_length == "year":
        if is_climate == True:
            df_snow = df[["snowdepth"]]
            df_resampled_snow = df_snow.resample("YS-AUG")
        
        df = df.drop("snowdepth", axis = 1, errors = "ignore")
        df_resampled = df.resample("YS")
            
    # Generic Aggregations (on everything but yearly snow)
    stats = df_resampled.agg(list(agg_funcs.values()))
    stats.columns = pd.MultiIndex.from_product([df.columns, list(agg_funcs.keys())]) # rename aggregations columns
    
    # Climate Sums (on all climate data but yearly snow)
    if is_climate and resample_length in ["week", "month", "season"]:
        stats[("precipitation", "sum")] = df_resampled[["precipitation"]].sum(min_count = 1)
        stats[("temperature", "gdd")] = df_resampled[["temperature"]].apply(lambda group: find_gdd(t = group))
        stats[("snowdepth", "gain")] = df_resampled[["snowdepth"]].apply(lambda group: find_snow_gain(s = group))
        stats[("snowdepth", "loss")] = df_resampled[["snowdepth"]].apply(lambda group: find_snow_loss(s = group))
    
    # Satellite AUCs (on just satellite data)
    if is_satellite:
        auc_gppe = df_resampled["gppe"].apply(calc_auc).apply(pd.Series)
        auc_gppe.columns = pd.MultiIndex.from_product([["gppe"], ["auc", "auc_std"]])
        auc_gpp = df_resampled["gpp"].apply(calc_auc).apply(pd.Series)
        auc_gpp.columns = pd.MultiIndex.from_product([["gpp"], ["auc", "auc_std"]])
        auc_ndvi = df_resampled["ndvi"].apply(calc_auc).apply(pd.Series)
        auc_ndvi.columns = pd.MultiIndex.from_product([["ndvi"], ["auc", "auc_std"]])
        stats = pd.concat([stats, auc_gppe, auc_gpp, auc_ndvi], axis = 1)
    
    # Warmth index (only seasonally and yearly)
    if resample_length in ["season", "year"] and is_climate == True:
        stats_wi = pd.DataFrame()
        stats_wi[("temperature", "wi")] = df_resampled[("temperature")].apply(lambda group: find_warmth_index(group, min_months = 3, period_mean = "MS"))
        stats = pd.concat([stats, stats_wi], axis = 1)
    
    # For yearly snow data
    if resample_length == "year" and is_climate == True:
        
        # Generic aggregations
        stats_snow = df_resampled_snow.agg(list(agg_funcs.values()))
        stats_snow.columns = pd.MultiIndex.from_product([df_snow.columns, list(agg_funcs.keys())]) # rename aggregations columns
        
        # Gain and loss
        stats_snow[("snowdepth", "gain")] = df_resampled_snow[["snowdepth"]].apply(lambda group: find_snow_gain(s = group))
        stats_snow[("snowdepth", "loss")] = df_resampled_snow[["snowdepth"]].apply(lambda group: find_snow_loss(s = group))
        
        # EOAS
        stats_snow[("snowdepth", "end-of-ablation-season")] = df_resampled_snow.apply(lambda group: find_end_of_ablation_season(group, "snowdepth"))
        stats_snow = stats_snow.shift(1, "YS") # by calculating on a resample starting mid-year (df_resampled_snow), the snow aggregates use the year of their beggining date which comes from the previous year. We shift it back to the current year.
        stats = pd.concat([stats, stats_snow], axis = 1)
        
    return stats

Calculate

In [ ]:
atmos.week = calc_aggregates(df = atmos.day, resample_length = "week")
atmos.month = calc_aggregates(df = atmos.day, resample_length = "month")
atmos.season = calc_aggregates(df = atmos.day, resample_length = "season")
atmos.annual = calc_aggregates(df = atmos.day, resample_length = "year")

for site, item in sites.items():
    item.sat_week = calc_aggregates(df = item.sat_day, resample_length = "week", is_satellite = True)
    item.sat_month = calc_aggregates(df = item.sat_day, resample_length = "month", is_satellite = True)
    item.sat_season = calc_aggregates(df = item.sat_day, resample_length = "season", is_satellite = True)
    item.sat_annual = calc_aggregates(df = item.sat_day, resample_length = "year", is_satellite = True)
    
for site, item in sites.items():
    item.cli_week = calc_aggregates(df = item.cli_day, resample_length = "week", is_climate = True)
    item.cli_month = calc_aggregates(df = item.cli_day, resample_length = "month", is_climate = True)
    item.cli_season = calc_aggregates(df = item.cli_day, resample_length = "season", is_climate = True)
    item.cli_annual = calc_aggregates(df = item.cli_day, resample_length = "year", is_climate = True)

Save

In [ ]:
with open(f"pkl/analysis_ready_sites.pkl", 'wb') as f:
        pickle.dump(sites, f)
with open(f"pkl/analysis_ready_atmos.pkl", 'wb') as f:
        pickle.dump(atmos, f)
with open(f"pkl/density.pkl", 'wb') as f:
        pickle.dump(density, f)

# Quick Load Data

In [ ]:
# Load Pickle Data
sites = None
with open("pkl/analysis_ready_sites.pkl", 'rb') as f:
    sites = pickle.load(f)
atmos = None
with open("pkl/analysis_ready_atmos.pkl", 'rb') as f:
    atmos = pickle.load(f)
density = None
with open("pkl/density.pkl", 'rb') as f:
    density = pickle.load(f)
model_stats = None
with open("pkl/model_stats.pkl", 'rb') as f:
    model_stats = pickle.load(f)


# Order the sites in the dictionaries
order = ["se_st1", "se_sto", "fi_ken", "fi_sod", "fi_var"]
res = dict()
for key in order:
    res[key] = sites[key]
sites = res

order = ["se_st1", "se_sto", "fi_ken", "fi_sod", "fi_var"]
res = dict()
for key in order:
    res[key] = density[key]
density = res

# Statistics

Make a dictionary to contain statistics

In [ ]:
stats_dict = {}
for key in sites:
    stats_dict[key] = {
        "coords": sites[key].coords,
        "elevation": sites[key].elevation
        }

Set global value for the number of bootstrap iterations

In [ ]:
global_n_boots = 10000

## Functions

In [ ]:
# Datetime to Decimal Years
def datetime_to_decimal_years(datetime_index):
    return datetime_index.year + (datetime_index.dayofyear - 1) / 365.25


# OLS Regression
def ols(x, y, n_boot_samples = False, random_state = 2):

    # OLS
    results = sm.OLS(y, sm.add_constant(x)).fit()
    x_range = np.linspace(x.min(), x.max(), 100)
    y_pred = results.predict(sm.add_constant(x_range))
    
    # Bootstrap CIs
    if n_boot_samples != False:
        rng = np.random.default_rng(seed = random_state)
        n = len(x)
        boot_preds = []
        boot_resids = []
        
        for _ in range(n_boot_samples):
            
            # Resample
            idx = rng.choice(n, size = n, replace = True)
            x_boot, y_boot = x.iloc[idx], y.iloc[idx]

            # Fit & Predict on idealised Xs (for smooth regression)
            results_boot = sm.OLS(y_boot, sm.add_constant(x_boot)).fit() # remember to use a different named variable inside the bootstrap loop
            y_pred_boot = results_boot.predict(sm.add_constant(x_range)) # use the same prediction range as above. Important! This makes all boot predictions use the same x range.
            
            # Fit & Predict on real Xs (for residual error bars)
            y_pred_boot_orginal = results_boot.predict(sm.add_constant(x)) # predict again but on original Xs so we can find y datapoints residuals. These are the actual y values of where the error is, not a relative error
            y_resid_boot = y - y_pred_boot_orginal
            
            # Store boot iterations
            boot_preds.append(y_pred_boot) # append pd.series to list
            boot_resids.append(y_resid_boot)
            
        # Collapse boot iterations to arrays
        boot_preds = np.array(boot_preds) # collapse list to np.array (converting from pd.series to np.array)
        boot_resids = pd.concat(boot_resids, axis = 1) # collapse to a df for this one so we can keep the index of the pd.series for merging/calcs with the original residuals later. Also doing sort false as apparently pandas likes to auto sort during a concat and our index is on sortable because dupicates (?). Very annoying, adding sort = false suppresses the warning.   sort = False
        
        # Y-pred Percentiles
        ci_lo = np.percentile(boot_preds, 5, axis = 0)
        ci_hi = np.percentile(boot_preds, 95, axis = 0)
        
        # Y-resid Percentiles
        resid_lo = boot_resids.quantile(0.05, axis = 1) # use quantile here as we are not working with np array but a pd.dataframe
        resid_hi = boot_resids.quantile(0.95, axis = 1)
        #resid_se = boot_resids.std(axis = 1) # not the right stat to use as we are looking at model uncertainty, not sample variation (???)
        
        # Return
        OLS_Stats = namedtuple("OLS_Stats", ["results", "x_range", "y_pred", "ci_lo", "ci_hi", "resid_lo", "resid_hi"])#, "resid_se"])
        return OLS_Stats(results, x_range, y_pred, ci_lo, ci_hi, resid_lo, resid_hi)#, resid_se)
    else: 
        OLS_Stats = namedtuple("OLS_Stats", ["results", "x_range", "y_pred"])
        return OLS_Stats(results, x_range, y_pred)

# Theil-sen
def theilsen(x, y, n_boot_samples = False, random_state = 2):
    
    # Theil-sen
    slope, intercept, slope_lo , slope_hi = theilslopes(y.copy(), x = x.copy(), alpha = 0.95, method = "separate") # using copy as it sorts the data
    x_range = np.linspace(x.min(), x.max(), 100)
    y_pred = intercept + slope * x_range
    
    # Bootstrap CIs
    if n_boot_samples != False:
        rng = np.random.default_rng(seed = random_state)
        n = len(x)
        boot_preds = []

        for _ in range(n_boot_samples):
            
            # Resample
            idx = rng.choice(n, size = n, replace = True)
            x_boot, y_boot = x.iloc[idx], y.iloc[idx]

            # Fit & Predict
            slope, intercept, *_  = theilslopes(y_boot.copy(), x = x_boot.copy(), alpha = 0.95, method = "separate") # using copy as it sorts the data
            y_pred_boot = intercept + slope * x_range # use the same prediction range as above. Important! This makes all boot predictions use the same x range.
            
            boot_preds.append(y_pred_boot)

        boot_preds = np.array(boot_preds)

        # Percentiles
        ci_lo = np.percentile(boot_preds, 10, axis = 0)
        ci_hi = np.percentile(boot_preds, 90, axis = 0)
        
        # Return
        TheilSen_Stats = namedtuple("TheilSen_Stats", ["slope", "intercept", "slope_lo", "slope_hi", "y_pred", "x_range", "ci_lo", "ci_hi"])
        return TheilSen_Stats(slope, intercept, slope_lo, slope_hi , y_pred, x_range, ci_lo, ci_hi)
    else: 
        TheilSen_Stats = namedtuple("TheilSen_Stats", ["slope", "intercept", "slope_lo", "slope_hi", "y_pred", "x_range"])
        return TheilSen_Stats(slope, intercept, slope_lo, slope_hi , y_pred, x_range)
    
# Kendall Trend Test
def kendall_trend(x, y):
    kt_tau, kt_pval = kendalltau(x, y)
    KT_Stats = namedtuple("KT_Stats", ["tau", "pval"])
    return KT_Stats(kt_tau, kt_pval)

## No. Unique Scenes

In [ ]:
df = pd.concat([sites[site].sat_day.copy() for site in sites], axis = 0)
print(f"Number of Unique Quality Satellite Scenes: {df["system:index"].nunique()}")

## GPPe Through Time

### Annual

In [ ]:
x_col = ("index", "")
y_col = ("gppe", "auc")
y_col_std = ("gppe", "auc_std")
var_names = ["index", "gppe", "std"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = item.sat_annual.copy()
    df.index = datetime_to_decimal_years(df.index)
    df = df.reset_index()
    df = df[[x_col, y_col, y_col_std]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]

    # Stats
    sample_std = np.std(df[y_var], axis = 0)
    ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    kt_stats = kendall_trend(df[x_var], df[y_var])
    ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    
    # Add entry to stats dict
    stats_dict[key]["gppe-time_annual"] = {
        "df": df,
        "x_var": var_names[0],
        "y_var": var_names[1],
        "y_var_std": var_names[2],
        "sample_std": sample_std,
        "ols": ols_stats,
        "kt": kt_stats,
        "ts": ts_stats
    }

In [ ]:
for idx, (key, item) in enumerate(stats_dict.items()):
    
    item = item["gppe-time_annual"]
    #ols_r2 = item["ols"].results.rsquared_adj.item()
    #ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
    ts_slope = item["ts"].slope
    kt_tau = item["kt"].tau
    kt_pval = item["kt"].pval
    
    print(f"At {key}, {item["y_var"]} over {item["x_var"]} reported {ts_slope:.2f} gCm⁻²a⁻¹ (τ:{kt_tau:.2f}, p:{kt_pval:.2f}). Sample StdDev: ±{item['sample_std'].item():.2f} (1σ).")
    
    
    # Check Data Distribution
    data = item["df"][item["y_var"]].squeeze()
    # Fit a normal distribution
    mu, std = norm.fit(data)
    # Generate x values for normal CDF
    x = np.linspace(data.min(), data.max(), 500)
    normal_cdf = norm.cdf(x, loc = mu, scale = std)


    # Test Plots
    fig, ax = plt.subplots(1, 2, figsize = (12, 4), layout = "constrained")
    ax = ax.flatten()
    
    # CDF
    ax[0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
    ax[0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
    ax[0].set_title("Normal Dist. CDF")
    
    # Data
    ax[1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
    ax[1].plot(item["ts"].x_range, item["ts"].y_pred, color = "C0")
    ax[1].fill_between(
        item["ts"].x_range.flatten(),
        np.array(item["ts"].ci_lo).flatten(),
        np.array(item["ts"].ci_hi).flatten(),
        color = "black",
        alpha = 0.1
    )
    ax[1].errorbar(
        item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
        yerr = item["df"][item["y_var_std"]].squeeze(),
        fmt = "o",
        elinewidth = 0.5,
        capsize = 2.5,
        color = "C0",
        alpha = 1
    )
    ax[1].set_title(key)
    ax[1].set_xlabel(item["x_var"])
    ax[1].set_ylabel(item["y_var"])
    plt.show()

### Season

In [ ]:
x_col = ("index", "")
y_col = ("gppe", "auc")
y_col_std = ("gppe", "auc_std")
var_names = ["index", "gppe", "std"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = sites[key].sat_season.copy()
    df[x_var] = datetime_to_decimal_years(df.index)
    df = df[[x_col, y_col, y_col_std]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]
    
    seasons = {
        "summer": df[df.index.month.isin([6, 7, 8])],
        #"winter": df[df.index.month.isin([12, 1, 2])], # winter has no gpp so no need to do stats or plot it
        "spring": df[df.index.month.isin([3, 4, 5])],
        "autumn": df[df.index.month.isin([9, 10, 11])]
    }
    
    stats_dict[key]["gppe-time_season"] = {}
    for season, df in seasons.items():

        # Stats
        sample_std = np.std(df[y_var], axis = 0)
        ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
        kt_stats = kendall_trend(df[x_var], df[y_var])
        ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
        
        # Add entry to stats dict
        stats_dict[key]["gppe-time_season"][season] = {
            "df": df,
            "x_var": var_names[0],
            "y_var": var_names[1],
            "y_var_std": var_names[2],
            "sample_std": sample_std,
            "ols": ols_stats,
            "kt": kt_stats,
            "ts": ts_stats
        }

In [ ]:
for site, entry in stats_dict.items():

    # Test Plots
    fig, ax = plt.subplots(3, 2, figsize = (12, 4*3), layout = "constrained")
    
    for idx, (season, item) in enumerate(entry["gppe-time_season"].items()):

        #ols_r2 = item["ols"].results.rsquared_adj.item()
        #ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
        ts_slope = item["ts"].slope
        n = item["y_var_std"].count
        kt_tau = item["kt"].tau
        kt_pval = item["kt"].pval
            
        print(f"At {key}, {item["y_var"]} over {item["x_var"]} reported {ts_slope:.2f} gCm⁻²a⁻¹ (τ:{kt_tau:.2f}, p:{kt_pval:.2f}). Sample StdDev: ±{item['sample_std'].item():.2f} (1σ).")
        
        # Check Data Distribution
        data = item["df"][item["y_var"]].squeeze()
        # Fit a normal distribution
        mu, std = norm.fit(data)
        # Generate x values for normal CDF
        x = np.linspace(data.min(), data.max(), 500)
        normal_cdf = norm.cdf(x, loc = mu, scale = std)

        
        # CDF
        ax[idx, 0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
        ax[idx, 0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
        ax[idx, 0].set_title("Normal Dist. CDF")
        
        # Data
        ax[idx, 1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
        ax[idx, 1].plot(item["ts"].x_range, item["ts"].y_pred, color = "C0")
        ax[idx, 1].fill_between(
            item["ts"].x_range.flatten(),
            np.array(item["ts"].ci_lo).flatten(),
            np.array(item["ts"].ci_hi).flatten(),
            color = "black",
            alpha = 0.1
        )
        ax[idx, 1].errorbar(
            item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
            yerr = item["df"][item["y_var_std"]].squeeze(),
            fmt = "o",
            elinewidth = 0.5,
            capsize = 2.5,
            color = "C0",
            alpha = 1
        )
        ax[idx, 1].set_title(key)
        ax[idx, 1].set_xlabel(item["x_var"])
        ax[idx, 1].set_ylabel(item["y_var"])
        ax[idx, 1].set_title(f"{season}: {item["y_var"]} over {item["x_var"]}")
        if idx == 3:
            ax[idx, 1].set_xlabel(item["x_var"])
        ax[idx, 1].set_ylabel(item["y_var"])

    plt.show()    
    print("")

## GPPe & NDVI Through Time

### Annual

GPPe

In [ ]:
x_col = ("index", "")
y_col = ("gppe", "auc")
var_names = ["index", "gppe"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = item.sat_annual.copy()
    df.index = datetime_to_decimal_years(df.index)
    df = df.reset_index()
    df = df[[x_col, y_col]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]
    
    # Z-score
    df[y_var] = zscore(df[y_var])

    # Stats
    sample_std = np.std(df[y_var], axis = 0)
    ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    kt_stats = kendall_trend(df[x_var], df[y_var])
    ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    
    # Add entry to stats dict
    stats_dict[key]["gppe_ndvi-time_annual"] = {}
    stats_dict[key]["gppe_ndvi-time_annual"]["gppe"] = {
        "df": df,
        "x_var": var_names[0],
        "y_var": var_names[1],
        "sample_std": sample_std,
        "ols": ols_stats,
        "kt": kt_stats,
        "ts": ts_stats
    }

In [ ]:
for idx, (key, item) in enumerate(stats_dict.items()):
    
    item = item["gppe_ndvi-time_annual"]["gppe"]
    #ols_r2 = item["ols"].results.rsquared_adj.item()
    #ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
    ts_slope = item["ts"].slope
    kt_tau = item["kt"].tau
    kt_pval = item["kt"].pval
    
    print(f"At {key}, {item["y_var"]} over {item["x_var"]} reported {ts_slope:.2f} Z-score (τ:{kt_tau:.2f}, p:{kt_pval:.2f})")
    
    
    # Check Data Distribution
    data = item["df"][item["y_var"]].squeeze()
    # Fit a normal distribution
    mu, std = norm.fit(data)
    # Generate x values for normal CDF
    x = np.linspace(data.min(), data.max(), 500)
    normal_cdf = norm.cdf(x, loc = mu, scale = std)


    # Test Plots
    fig, ax = plt.subplots(1, 2, figsize = (12, 4), layout = "constrained")
    ax = ax.flatten()
    
    # CDF
    ax[0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
    ax[0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
    ax[0].set_title("Normal Dist. CDF")
    
    # Data
    ax[1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
    plt.plot(item["ts"].x_range, item["ts"].y_pred, color = "C0")
    ax[1].fill_between(
        item["ts"].x_range.flatten(),
        np.array(item["ts"].ci_lo).flatten(),
        np.array(item["ts"].ci_hi).flatten(),
        color = "black",
        alpha = 0.1
    )
    #ax[1].errorbar(
    #    item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
    #    yerr = item["df"][item["y_var_std"]].squeeze(),
    #    fmt = "o",
    #    elinewidth = 0.5,
    #    capsize = 2.5,
    #    color = "C0",
    #    alpha = 1
    #)
    ax[1].set_title(key)
    ax[1].set_xlabel(item["x_var"])
    ax[1].set_ylabel(item["y_var"])
    plt.show()

NDVI

In [ ]:
x_col = ("index", "")
y_col = ("ndvi", "auc")
var_names = ["index", "ndvi"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = item.sat_annual.copy()
    df.index = datetime_to_decimal_years(df.index)
    df = df.reset_index()
    df = df[[x_col, y_col]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]
    
    # Z-score
    df[y_var] = zscore(df[y_var])

    # Stats
    sample_std = np.std(df[y_var], axis = 0)
    ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    kt_stats = kendall_trend(df[x_var], df[y_var])
    ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    
    # Add entry to stats dict
    #stats_dict[key]["gppe_ndvi-time_annual"] = {} # already made when calculaitng gppe stats
    stats_dict[key]["gppe_ndvi-time_annual"]["ndvi"] = {
        "df": df,
        "x_var": var_names[0],
        "y_var": var_names[1],
        "sample_std": sample_std,
        "ols": ols_stats,
        "kt": kt_stats,
        "ts": ts_stats
    }

In [ ]:
for idx, (key, item) in enumerate(stats_dict.items()):
    
    item = item["gppe_ndvi-time_annual"]["ndvi"]
    #ols_r2 = item["ols"].results.rsquared_adj.item()
    #ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
    ts_slope = item["ts"].slope
    kt_tau = item["kt"].tau
    kt_pval = item["kt"].pval
    
    print(f"At {key}, {item["y_var"]} over {item["x_var"]} reported {ts_slope:.2f} Z-score (τ:{kt_tau:.2f}, p:{kt_pval:.2f})")
    
    
    # Check Data Distribution
    data = item["df"][item["y_var"]].squeeze()
    # Fit a normal distribution
    mu, std = norm.fit(data)
    # Generate x values for normal CDF
    x = np.linspace(data.min(), data.max(), 500)
    normal_cdf = norm.cdf(x, loc = mu, scale = std)


    # Test Plots
    fig, ax = plt.subplots(1, 2, figsize = (12, 4), layout = "constrained")
    ax = ax.flatten()
    
    # CDF
    ax[0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
    ax[0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
    ax[0].set_title("Normal Dist. CDF")
    
    # Data
    ax[1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
    plt.plot(item["ts"].x_range, item["ts"].y_pred, color = "C0")
    ax[1].fill_between(
        item["ts"].x_range.flatten(),
        np.array(item["ts"].ci_lo).flatten(),
        np.array(item["ts"].ci_hi).flatten(),
        color = "black",
        alpha = 0.1
    )
    #ax[1].errorbar(
    #    item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
    #    yerr = item["df"][item["y_var_std"]].squeeze(),
    #    fmt = "o",
    #    elinewidth = 0.5,
    #    capsize = 2.5,
    #    color = "C0",
    #    alpha = 1
    #)
    ax[1].set_title(key)
    ax[1].set_xlabel(item["x_var"])
    ax[1].set_ylabel(item["y_var"])
    plt.show()

### Peak

GPPe

In [ ]:
x_col = ("index", "")
y_col = ("gppe", "max")
var_names = ["index", "gppe"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = item.sat_annual.copy()
    df.index = datetime_to_decimal_years(df.index)
    df = df.reset_index()
    df = df[[x_col, y_col]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]
    
    # Z-score
    df[y_var] = zscore(df[y_var])

    # Stats
    sample_std = np.std(df[y_var], axis = 0)
    ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    kt_stats = kendall_trend(df[x_var], df[y_var])
    ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    
    # Add entry to stats dict
    stats_dict[key]["gppe_ndvi-time_peak"] = {} # already made when calculaitng gppe stats
    stats_dict[key]["gppe_ndvi-time_peak"]["gppe"] = {
        "df": df,
        "x_var": var_names[0],
        "y_var": var_names[1],
        "sample_std": sample_std,
        "ols": ols_stats,
        "kt": kt_stats,
        "ts": ts_stats
    }

In [ ]:
for idx, (key, item) in enumerate(stats_dict.items()):
    
    item = item["gppe_ndvi-time_peak"]["gppe"]
    #ols_r2 = item["ols"].results.rsquared_adj.item()
    #ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
    ts_slope = item["ts"].slope
    kt_tau = item["kt"].tau
    kt_pval = item["kt"].pval
    
    print(f"At {key}, {item["y_var"]} over {item["x_var"]} reported {ts_slope:.2f} Z-score (τ:{kt_tau:.2f}, p:{kt_pval:.2f})")
    
    
    # Check Data Distribution
    data = item["df"][item["y_var"]].squeeze()
    # Fit a normal distribution
    mu, std = norm.fit(data)
    # Generate x values for normal CDF
    x = np.linspace(data.min(), data.max(), 500)
    normal_cdf = norm.cdf(x, loc = mu, scale = std)


    # Test Plots
    fig, ax = plt.subplots(1, 2, figsize = (12, 4), layout = "constrained")
    ax = ax.flatten()
    
    # CDF
    ax[0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
    ax[0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
    ax[0].set_title("Normal Dist. CDF")
    
    # Data
    ax[1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
    plt.plot(item["ts"].x_range, item["ts"].y_pred, color = "C0")
    ax[1].fill_between(
        item["ts"].x_range.flatten(),
        np.array(item["ts"].ci_lo).flatten(),
        np.array(item["ts"].ci_hi).flatten(),
        color = "black",
        alpha = 0.1
    )
    #ax[1].errorbar(
    #    item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
    #    yerr = item["df"][item["y_var_std"]].squeeze(),
    #    fmt = "o",
    #    elinewidth = 0.5,
    #    capsize = 2.5,
    #    color = "C0",
    #    alpha = 1
    #)
    ax[1].set_title(key)
    ax[1].set_xlabel(item["x_var"])
    ax[1].set_ylabel(item["y_var"])
    plt.show()

NDVI

In [ ]:
x_col = ("index", "")
y_col = ("ndvi", "max")
var_names = ["index", "ndvi"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = item.sat_annual.copy()
    df.index = datetime_to_decimal_years(df.index)
    df = df.reset_index()
    df = df[[x_col, y_col]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]
    
    # Z-score
    df[y_var] = zscore(df[y_var])

    # Stats
    sample_std = np.std(df[y_var], axis = 0)
    ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    kt_stats = kendall_trend(df[x_var], df[y_var])
    ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    
    # Add entry to stats dict
    #stats_dict[key]["gppe_ndvi-time_peak"] = {} # already made when calculaitng gppe stats
    stats_dict[key]["gppe_ndvi-time_peak"]["ndvi"] = {
        "df": df,
        "x_var": var_names[0],
        "y_var": var_names[1],
        "sample_std": sample_std,
        "ols": ols_stats,
        "kt": kt_stats,
        "ts": ts_stats
    }

In [ ]:
for idx, (key, item) in enumerate(stats_dict.items()):
    
    item = item["gppe_ndvi-time_peak"]["ndvi"]
    #ols_r2 = item["ols"].results.rsquared_adj.item()
    #ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
    ts_slope = item["ts"].slope
    kt_tau = item["kt"].tau
    kt_pval = item["kt"].pval
    
    print(f"At {key}, {item["y_var"]} over {item["x_var"]} reported {ts_slope:.2f} Z-score (τ:{kt_tau:.2f}, p:{kt_pval:.2f})")
    
    
    # Check Data Distribution
    data = item["df"][item["y_var"]].squeeze()
    # Fit a normal distribution
    mu, std = norm.fit(data)
    # Generate x values for normal CDF
    x = np.linspace(data.min(), data.max(), 500)
    normal_cdf = norm.cdf(x, loc = mu, scale = std)


    # Test Plots
    fig, ax = plt.subplots(1, 2, figsize = (12, 4), layout = "constrained")
    ax = ax.flatten()
    
    # CDF
    ax[0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
    ax[0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
    ax[0].set_title("Normal Dist. CDF")
    
    # Data
    ax[1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
    plt.plot(item["ts"].x_range, item["ts"].y_pred, color = "C0")
    ax[1].fill_between(
        item["ts"].x_range.flatten(),
        np.array(item["ts"].ci_lo).flatten(),
        np.array(item["ts"].ci_hi).flatten(),
        color = "black",
        alpha = 0.1
    )
    #ax[1].errorbar(
    #    item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
    #    yerr = item["df"][item["y_var_std"]].squeeze(),
    #    fmt = "o",
    #    elinewidth = 0.5,
    #    capsize = 2.5,
    #    color = "C0",
    #    alpha = 1
    #)
    ax[1].set_title(key)
    ax[1].set_xlabel(item["x_var"])
    ax[1].set_ylabel(item["y_var"])
    plt.show()

## GPPe and Temperature (WI)

### Annual

In [ ]:
x_col = ("temperature", "wi")
y_col = ("gppe", "auc")
y_col_std = ("gppe", "auc_std")
var_names = ["temperature", "gppe", "std"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = pd.merge(left = item.sat_annual.copy(), right = item.cli_annual.copy(), left_index = True, right_index = True, how = "inner")
    #df.index = datetime_to_decimal_years(df.index)
    #df = df.reset_index()
    df = df[[x_col, y_col, y_col_std]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]

    # Stats
    sample_std = np.std(df[y_var], axis = 0)
    ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    #kt_stats = kendall_trend(df[x_var], df[y_var])
    #ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    
    # Add entry to stats dict
    stats_dict[key]["gppe-wi_annual"] = {
        "df": df,
        "x_var": var_names[0],
        "y_var": var_names[1],
        "y_var_std": var_names[2],
        "sample_std": sample_std,
        "ols": ols_stats,
        #"kt": kt_stats,
        #"ts": ts_stats
    }

In [ ]:
for idx, (key, item) in enumerate(stats_dict.items()):
    
    item = item["gppe-wi_annual"]
    ols_r2 = item["ols"].results.rsquared_adj.item()
    ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
    #ts_slope = item["ts"].slope
    #kt_tau = item["kt"].tau
    #kt_pval = item["kt"].pval
    
    print(f"At {key}, {item["y_var"]} over {item["x_var"]} reported (adj.r2:{ols_r2:.2f}, p:{ols_pval:.2f})")
     
    print(item["ols"].results.summary())
    
    # Check Data Distribution
    data = item["df"][item["y_var"]].squeeze()
    # Fit a normal distribution
    mu, std = norm.fit(data)
    # Generate x values for normal CDF
    x = np.linspace(data.min(), data.max(), 500)
    normal_cdf = norm.cdf(x, loc = mu, scale = std)


    # Test Plots
    fig, ax = plt.subplots(1, 2, figsize = (12, 4), layout = "constrained")
    ax = ax.flatten()
    
    # CDF
    ax[0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
    ax[0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
    ax[0].set_title("Normal Dist. CDF")
    
    # Data
    ax[1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
    ax[1].plot(item["ols"].x_range, item["ols"].results.predict(sm.add_constant(item["ols"].x_range)))
    ax[1].fill_between(item["ols"].x_range.flatten(), item["ols"].ci_lo, item["ols"].ci_hi, alpha = 0.1, color = "black", lw = 0)
    ax[1].errorbar(
        item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
        yerr = item["df"][item["y_var_std"]].squeeze(),
        fmt = "o",
        elinewidth = 0.5,
        capsize = 2.5,
        color = "C0",
        alpha = 1
    )
    ax[1].set_title(key)
    ax[1].set_xlabel(item["x_var"])
    ax[1].set_ylabel(item["y_var"])
    plt.show()

### Season

In [ ]:
x_col = ("temperature", "wi")
y_col = ("gppe", "auc")
y_col_std = ("gppe", "auc_std")
var_names = ["temperature", "gppe", "std"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = pd.merge(left = item.sat_season.copy(), right = item.cli_season.copy(), left_index = True, right_index = True, how = "inner")
    #df.index = datetime_to_decimal_years(df.index)
    #df = df.reset_index()
    df = df[[x_col, y_col, y_col_std]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]
    y_var_std = var_names[2]
    
    seasons = {
        "summer": df[df.index.month.isin([6, 7, 8])],
        #"winter": df[df.index.month.isin([12, 1, 2])], # no gppe in winter!
        "spring": df[df.index.month.isin([3, 4, 5])],
        "autumn": df[df.index.month.isin([9, 10, 11])]
    }
    
    stats_dict[key]["gppe-wi_season"] = {}
    for season, df in seasons.items():

        # Stats
        sample_std = np.std(df[y_var], axis = 0)
        ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
        #kt_stats = kendall_trend(df[x_var], df[y_var])
        #ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
        
        # Add entry to stats dict
        stats_dict[key]["gppe-wi_season"][season] = {
            "df": df,
            "x_var": var_names[0],
            "y_var": var_names[1],
            "y_var_std": var_names[2],
            "sample_std": sample_std,
            "ols": ols_stats,
            #"kt": kt_stats,
            #"ts": ts_stats
        }

In [ ]:
for site, entry in stats_dict.items():

    # Test Plots
    fig, ax = plt.subplots(4, 2, figsize = (12, 4*4), layout = "constrained")
    
    for idx, (season, item) in enumerate(entry["gppe-wi_season"].items()):
        
        ols_r2 = item["ols"].results.rsquared_adj.item()
        ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
        #ts_slope = item["ts"].slope
        #n = item["y_std"].count
        #kt_tau = item["kt"].tau
        #kt_pval = item["kt"].pval
            
        print(f"At {site} in {season}, {item["y_var"]} over {item["x_var"]} reported (adj.r2:{ols_r2:.2f}, p:{ols_pval:.2f})")
        
        print(item["ols"].results.summary())
        print("")
        
        # Check Data Distribution
        data = item["df"][item["y_var"]].squeeze()
        # Fit a normal distribution
        mu, std = norm.fit(data)
        # Generate x values for normal CDF
        x = np.linspace(data.min(), data.max(), 500)
        normal_cdf = norm.cdf(x, loc = mu, scale = std)

        
        # CDF
        ax[idx, 0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
        ax[idx, 0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
        ax[idx, 0].set_title(f"{season}: Normal Dist. CDF")
        
        # Data
        ax[idx, 1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
        ax[idx, 1].plot(item["ols"].x_range, item["ols"].results.predict(sm.add_constant(item["ols"].x_range)))
        ax[idx, 1].fill_between(item["ols"].x_range.flatten(), item["ols"].ci_lo, item["ols"].ci_hi, alpha = 0.1, color = "black", lw = 0)
        ax[idx, 1].errorbar(
            item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
            yerr = item["df"][item["y_var_std"]].squeeze(),
            fmt = "o",
            elinewidth = 0.5,
            capsize = 2.5,
            color = "C0",
            alpha = 1
        )
        ax[idx, 1].set_title(f"{season}: {item["y_var"]} over {item["x_var"]}")
        if idx == 3:
            ax[idx, 1].set_xlabel(item["x_var"])
        ax[idx, 1].set_ylabel(item["y_var"])
    
    plt.show()
    print("")

## GPPe Temperature (WI) Residuals and Snowdepth

### Residuals
Take the residuals from the gppe - temperature WI relationship (we rerun the OLS so we have the option to look at sub-seasonal relationships too).

Define residuals function

In [ ]:
def residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy, var_w_tup = None, debug = False):
    
    # Merge by outer index
    df = pd.merge(
        df_x,
        df_y,
        left_index = True,
        right_index = True,
        how = "outer",
    )
    
    # Get vars
    df_res = df[[var_x_tup, var_y_tup]].copy()
    df_res.dropna(inplace = True)
    
    # Calc Regression
    if logy:
        ols_stats = ols(df_res[var_x_tup], np.log1p(df_res[var_y_tup]), n_boot_samples = global_n_boots)
        resid = np.exp(ols_stats.results.resid) # exponentiate residuals
        resid_lo = np.exp(ols_stats.resid_lo)
        resid_hi = np.exp(ols_stats.resid_hi)
    else:
        ols_stats = ols(df_res[var_x_tup], df_res[var_y_tup], n_boot_samples = global_n_boots)
        resid = ols_stats.results.resid
        resid_lo = ols_stats.resid_lo
        resid_hi = ols_stats.resid_hi
    
    resid.name = var_z_tup # name into multilevel header
    resid_lo.name = (var_z_tup[0], var_z_tup[1] + "_lo") # name into multilevel header
    resid_hi.name = (var_z_tup[0], var_z_tup[1] + "_hi") # name into multilevel header
    resid_df = pd.concat([resid, resid_lo, resid_hi], axis = 1)
    
    # Merge resid back into y df
    df_y = pd.merge(
        df_y,
        resid_df,
        left_index = True,
        right_index = True,
        how = "outer",
    )
    
    if debug:
        # Residuals
        plt.scatter(df_res[var_x_tup], df_res[var_y_tup])
        pred_y = results.get_prediction(X_range) # Predict
        if logy:
            pred_summary = pred_y.summary_frame(alpha = 0.1)
            pred_y = np.exp(pred_summary['mean']) # Exponentiate predictions
            ci_lower = np.exp(pred_summary['mean_ci_lower'])
            ci_upper = np.exp(pred_summary['mean_ci_upper'])
        else:
            pred_summary = pred_y.summary_frame(alpha = 0.1)
            pred_y = pred_summary['mean']
            ci_lower = pred_summary['mean_ci_lower']
            ci_upper = pred_summary['mean_ci_upper']
            
        plt.plot(x_range, pred_y, color = "black", lw = 0.8, label = "Exponential fit")
        plt.fill_between(x_range, ci_lower, ci_upper, color = "black", alpha = 0.1, lw = 0)
        plt.show()
        
        # Residuals against W
        df = pd.merge(
            df_x,
            df_y,
            left_index = True,
            right_index = True,
            how = "outer",
        )

        df = df[[var_w_tup, var_z_tup]].copy()
        df.dropna(inplace = True)
        df = df[df.index.month.isin([3, 4, 5])]
        
        if logy:
            results, ci_upper, ci_lower, predictions, x_range, X_range, *_, = ols(df[var_w_tup], np.log1p(df[var_z_tup]))
            resid = np.exp(results.resid) # exponentiate residuals
        else:
            results, ci_upper, ci_lower, predictions, x_range, X_range, *_, = ols(df[var_w_tup], df[var_z_tup])
            resid = results.resid # exponentiate residuals
        
        pred_y = results.get_prediction(X_range) # Predict
        if logy:
            pred_summary = pred_y.summary_frame(alpha = 0.1)
            pred_y = np.exp(pred_summary['mean']) # Exponentiate predictions
            ci_lower = np.exp(pred_summary['mean_ci_lower'])
            ci_upper = np.exp(pred_summary['mean_ci_upper'])
        else:
            pred_summary = pred_y.summary_frame(alpha = 0.1)
            pred_y = pred_summary['mean']
            ci_lower = pred_summary['mean_ci_lower']
            ci_upper = pred_summary['mean_ci_upper']

        plt.scatter(df[var_w_tup], df[var_z_tup])
        plt.plot(x_range, pred_y, color = "black", lw = 0.8, label = "Exponential fit")
        plt.fill_between(x_range, ci_lower, ci_upper, color = "black", alpha = 0.1, lw = 0)
        plt.show()
        
    return df_y

In [ ]:
for key in sites:
    
    ## Basic Temp
    ## Day
    #df_x = sites[key].cli_day
    #df_y = sites[key].sat_day
    #var_x_tup = ("temperature")
    #var_y_tup = ("gppe")
    #var_z_tup = ("gppe_res_mean")
    #sites[key].sat_day = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = True, debug = False)
    ## Week
    #df_x = sites[key].cli_week
    #df_y = sites[key].sat_week
    #var_x_tup = ("temperature", "mean")
    #var_y_tup = ("gppe", "auc")
    #var_z_tup = ("gppe", "auc_res_mean")
    #sites[key].sat_week = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = True, debug = False)
    ## Month
    #df_x = sites[key].cli_month
    #df_y = sites[key].sat_month
    #var_x_tup = ("temperature", "mean")
    #var_y_tup = ("gppe", "auc")
    #var_z_tup = ("gppe", "auc_res_mean")
    #sites[key].sat_month = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = True, debug = False)
    ## Season
    #df_x = sites[key].cli_season
    #df_y = sites[key].sat_season
    #var_x_tup = ("temperature", "mean")
    #var_y_tup = ("gppe", "auc")
    #var_z_tup = ("gppe", "auc_res_mean")
    #sites[key].sat_season = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = False, debug = False)
    ## Annual
    #df_x = sites[key].cli_annual
    #df_y = sites[key].sat_annual
    #var_x_tup = ("temperature", "mean")
    #var_y_tup = ("gppe", "auc")
    #var_z_tup = ("gppe", "auc_res_mean")
    #sites[key].sat_annual = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = False, debug = False)
    
    # Warmth Index
    # Season
    df_x = sites[key].cli_season
    df_y = sites[key].sat_season
    var_x_tup = ("temperature", "wi")
    var_y_tup = ("gppe", "auc")
    var_z_tup = ("gppe", "auc_res_wi")
    sites[key].sat_season = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = False, debug = False)
    # Annual
    df_x = sites[key].cli_annual
    df_y = sites[key].sat_annual
    var_x_tup = ("temperature", "wi")
    var_y_tup = ("gppe", "auc")
    var_z_tup = ("gppe", "auc_res_wi")
    sites[key].sat_annual = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = False, debug = False)
    
    ## GDD
    ## Day
    #df_x = sites[key].cli_day
    #df_y = sites[key].sat_day
    #var_x_tup = ("temperature")
    #var_y_tup = ("gppe")
    #var_z_tup = ("gppe_res_gdd")
    #sites[key].sat_day = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = True, debug = False)
    ## Week
    #df_x = sites[key].cli_week
    #df_y = sites[key].sat_week
    #var_x_tup = ("temperature", "gdd")
    #var_y_tup = ("gppe", "auc")
    #var_z_tup = ("gppe", "auc_res_gdd")
    #sites[key].sat_week = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = True, debug = False)
    ## Month
    #df_x = sites[key].cli_month
    #df_y = sites[key].sat_month
    #var_x_tup = ("temperature", "gdd")
    #var_y_tup = ("gppe", "auc")
    #var_z_tup = ("gppe", "auc_res_gdd")
    #sites[key].sat_month = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = True, debug = False)
    ## Season
    #df_x = sites[key].cli_season
    #df_y = sites[key].sat_season
    #var_x_tup = ("temperature", "gdd")
    #var_y_tup = ("gppe", "auc")
    #var_z_tup = ("gppe", "auc_res_gdd")
    #sites[key].sat_season = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = False, debug = False)
    ## Annual
    #df_x = sites[key].cli_annual
    #df_y = sites[key].sat_annual
    #var_x_tup = ("temperature", "gdd")
    #var_y_tup = ("gppe", "auc")
    #var_z_tup = ("gppe", "auc_res_gdd")
    #sites[key].sat_annual = residuals(df_x, df_y, var_x_tup, var_y_tup, var_z_tup, logy = False, debug = False)

### Annual

In [ ]:
x_col = ("snowdepth", "mean")
y_col = ("gppe", "auc_res_wi")
y_col_lo = ("gppe", "auc_res_wi_lo")
y_col_hi = ("gppe", "auc_res_wi_hi")
var_names = ["snowdepth", "gppe", "lo", "hi"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = pd.merge(left = item.sat_annual.copy(), right = item.cli_annual.copy(), left_index = True, right_index = True, how = "inner")
    #df.index = datetime_to_decimal_years(df.index)
    #df = df.reset_index()
    df = df[[x_col, y_col, y_col_lo, y_col_hi]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]

    # Stats
    sample_std = np.std(df[y_var], axis = 0)
    ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    #kt_stats = kendall_trend(df[x_var], df[y_var])
    #ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
    
    # Add entry to stats dict
    stats_dict[key]["gppe-snow_annual"] = {
        "df": df,
        "x_var": var_names[0],
        "y_var": var_names[1],
        "y_var_lo": var_names[2],
        "y_var_hi": var_names[3],
        "sample_std": sample_std,
        "ols": ols_stats,
        #"kt": kt_stats,
        #"ts": ts_stats
    }

In [ ]:
for idx, (key, item) in enumerate(stats_dict.items()):
    
    item = item["gppe-snow_annual"]
    ols_r2 = item["ols"].results.rsquared_adj.item()
    ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
    #ts_slope = item["ts"].slope
    #kt_tau = item["kt"].tau
    #kt_pval = item["kt"].pval
    
    print(f"At {key}, {item["y_var"]} over {item["x_var"]} reported (adj.r2:{ols_r2:.2f}, p:{ols_pval:.2f})")
    
    print(item["ols"].results.summary())
    print("")
     
    
    # Check Data Distribution
    data = item["df"][item["y_var"]].squeeze()
    # Fit a normal distribution
    mu, std = norm.fit(data)
    # Generate x values for normal CDF
    x = np.linspace(data.min(), data.max(), 500)
    normal_cdf = norm.cdf(x, loc = mu, scale = std)


    # Test Plots
    fig, ax = plt.subplots(1, 2, figsize = (12, 4), layout = "constrained")
    ax = ax.flatten()
    
    # CDF
    ax[0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
    ax[0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
    ax[0].set_title("Normal Dist. CDF")
    
    # Data
    ax[1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
    ax[1].plot(item["ols"].x_range, item["ols"].results.predict(sm.add_constant(item["ols"].x_range)))
    ax[1].fill_between(item["ols"].x_range.flatten(), item["ols"].ci_lo, item["ols"].ci_hi, alpha = 0.1, color = "black", lw = 0)
    ax[1].errorbar(
        item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
        yerr = np.array([(item["df"][item["y_var"]].values - item["df"][item["y_var_lo"]].values).flatten(), (item["df"][item["y_var_hi"]].values - item["df"][item["y_var"]].values).flatten()]),#np.array([pd.Series((item["df"][item["y_var"]].values - item["df"][item["y_var_lo"]].values).flatten(), index = item["df"].index), pd.Series((item["df"][item["y_var_hi"]].values - item["df"][item["y_var"]].values).flatten(), index = item["df"].index)]), # pandas cols are different names so cannot subract one anotger. so take values and then make back into pd.series
        fmt = "o",
        elinewidth = 0.5,
        capsize = 2.5,
        color = "C0",
        alpha = 1
    )
    ax[1].set_title(key)
    ax[1].set_xlabel(item["x_var"])
    ax[1].set_ylabel(item["y_var"])
    plt.show()
    

### Season

Lag function

In [ ]:
def lag(df, lag, month_filter):
    df[x_var] = df[x_var].shift(lag)
    df.dropna(inplace = True)
    df = df[df.index.month.isin(month_filter)]
    return df

In [ ]:
x_col = ("snowdepth", "mean")
y_col = ("gppe", "auc_res_wi")
y_col_lo = ("gppe", "auc_res_wi_lo")
y_col_hi = ("gppe", "auc_res_wi_hi")
var_names = ["snowdepth", "gppe", "lo", "hi"]

for idx, (key, item) in enumerate(sites.items()):
    
    # Get Data
    df = pd.merge(left = item.sat_season.copy(), right = item.cli_season.copy(), left_index = True, right_index = True, how = "inner")
    #df.index = datetime_to_decimal_years(df.index)
    #df = df.reset_index()
    df = df[[x_col, y_col, y_col_lo, y_col_hi]]
    df = df.droplevel(level = 0, axis = 1)
    df.columns = [var_names]
    df.dropna(inplace = True)
    x_var = var_names[0] # drop 2nd item in var tuples
    y_var = var_names[1]
    y_var_std = var_names[2]
    
    seasons = {
        #"summer": df[df.index.month.isin([6, 7, 8])], # no snow in summer
        #"winter": df[df.index.month.isin([12, 1, 2])], # no gppe in winter
        "spring": df[df.index.month.isin([3, 4, 5])],
        "spring lag": lag(df.copy(), 1, [3, 4, 5]), # must be copy as shift is an in place function
        "autumn": df[df.index.month.isin([9, 10, 11])]
    }
    
    stats_dict[key]["gppe-snow_season"] = {}
    for season, df in seasons.items():

        # Stats
        sample_std = np.std(df[y_var], axis = 0)
        ols_stats = ols(df[x_var], df[y_var], n_boot_samples = global_n_boots)
        #kt_stats = kendall_trend(df[x_var], df[y_var])
        #ts_stats = theilsen(df[x_var], df[y_var], n_boot_samples = global_n_boots)
        
        # Add entry to stats dict
        stats_dict[key]["gppe-snow_season"][season] = {
            "df": df,
            "x_var": var_names[0],
            "y_var": var_names[1],
            "y_var_lo": var_names[2],
            "y_var_hi": var_names[3],
            "sample_std": sample_std,
            "ols": ols_stats,
            #"kt": kt_stats,
            #"ts": ts_stats
        }

In [ ]:
for site, entry in stats_dict.items():

    # Test Plots
    fig, ax = plt.subplots(3, 2, figsize = (12, 4*3), layout = "constrained")
    
    for idx, (season, item) in enumerate(entry["gppe-snow_season"].items()):
        
        ols_r2 = item["ols"].results.rsquared_adj.item()
        ols_pval = item["ols"].results.pvalues[(item["x_var"]),].item()
        #ts_slope = item["ts"].slope
        #n = item["y_std"].count
        #kt_tau = item["kt"].tau
        #kt_pval = item["kt"].pval
            
        print(f"At {site} in {season}, {item["y_var"]} over {item["x_var"]} reported (adj.r2:{ols_r2:.2f}, p:{ols_pval:.2f})")
        
        print(item["ols"].results.summary())
        print("")
        
        # Check Data Distribution
        data = item["df"][item["y_var"]].squeeze()
        # Fit a normal distribution
        mu, std = norm.fit(data)
        # Generate x values for normal CDF
        x = np.linspace(data.min(), data.max(), 500)
        normal_cdf = norm.cdf(x, loc = mu, scale = std)

        
        # CDF
        ax[idx, 0].ecdf(data, linestyle = "", marker = ".", color = "C0") # Empirical CDF
        ax[idx, 0].plot(x, normal_cdf, color = "C3", linewidth = 1.5) # Normal CDF line
        ax[idx, 0].set_title(f"{season}: Normal Dist. CDF")
        
        # Data
        ax[idx, 1].scatter(item["df"][item["x_var"]], item["df"][item["y_var"]])
        ax[idx, 1].plot(item["ols"].x_range, item["ols"].results.predict(sm.add_constant(item["ols"].x_range)))
        ax[idx, 1].fill_between(item["ols"].x_range.flatten(), item["ols"].ci_lo, item["ols"].ci_hi, alpha = 0.1, color = "black", lw = 0)
        ax[idx, 1].errorbar(
            item["df"][item["x_var"]].squeeze(), item["df"][item["y_var"]].squeeze(),
            yerr = np.array([pd.Series((item["df"][item["y_var"]].values - item["df"][item["y_var_lo"]].values).flatten(), index = item["df"].index), pd.Series((item["df"][item["y_var_hi"]].values - item["df"][item["y_var"]].values).flatten(), index = item["df"].index)]), # pandas cols are different names so cannot subract one anotger. so take values and then make back into pd.series
            fmt = "o",
            elinewidth = 0.5,
            capsize = 2.5,
            color = "C0",
            alpha = 1
        )
        ax[idx, 1].set_title(f"{season}: {item["y_var"]} over {item["x_var"]}")
        if idx == 3:
            ax[idx, 1].set_xlabel(item["x_var"])
        ax[idx, 1].set_ylabel(item["y_var"])
    
    plt.show()
    print("")

## GPPe Phenology

### All Sites

In [ ]:
y_vars = ["gpp", "gppe", "ndvi"]

# Get Data
df_day = pd.concat([sites[site].sat_day.copy() for site in sites], axis = 0)
df_day.index = df_day.index.day_of_year # Index to day of year
df_day["week"] = pd.cut(df_day.index, bins = np.arange(0, 366, 7), right = False, labels = False) # Add Week No. Column

df_week = []
for var in y_vars:
    
    # Z-score
    df_day[var] = zscore(df_day[var], nan_policy = "omit")
    
    # Group by Week
    df_week_loop = df_day[[var, "week"]].groupby("week")[var].agg(["mean", lambda x: np.nanpercentile(x, 25), lambda x: np.nanpercentile(x, 75)])
    df_week_loop.columns = pd.MultiIndex.from_tuples([(var, "mean"), (var, "q25"), (var, "q75")]) # Make multi-index col names
   
    df_week.append(df_week_loop)

df_week = pd.concat(df_week, axis = 1)

# X positions
df_week.index = df_week.index.to_numpy() * 7 + 3  # Mid-point of week

# Stats
#mean_columns = df_week.loc[:, (slice(None), 'mean')] # just the means
#
#levene_res = levene(*[mean_columns[col] for col in mean_columns])
#print("Levene:", levene_res.statistic, levene_res.pvalue)
#
#anova_res = f_oneway(*[mean_columns[col] for col in mean_columns]) # unpack from list comprehension to pass as individual arguments
#print("ANOVA F-statistic:", anova_res.statistic, anova_res.pvalue)
#
## Store
stats_dict_all = {
    "phenology": {
        "df_day": df_day,
        "df_week": df_week,
        "y_vars": y_vars,
#        "levene": levene_res,
#        "anova": anova_res
    }
}

### Individual Sites

In [ ]:
y_vars = ["gpp", "gppe", "ndvi"]

for idx, (key, item) in enumerate(sites.items()):

    # Get Data
    df_day = item.sat_day.copy()

    df_day.index = df_day.index.day_of_year # Index to day of year
    df_day["week"] = pd.cut(df_day.index, bins = np.arange(0, 366, 7), right = False, labels = False) # Add Week No. Column

    df_week = []
    for var in y_vars:
        
        # Z-score
        df_day[var] = zscore(df_day[var], nan_policy = "omit")
        
        # Group by Week
        df_week_loop = df_day[[var, "week"]].groupby("week")[var].agg(["mean", lambda x: np.nanpercentile(x, 25), lambda x: np.nanpercentile(x, 75)])
        df_week_loop.columns = pd.MultiIndex.from_tuples([(var, "mean"), (var, "q25"), (var, "q75")]) # Make multi-index col names
    
        df_week.append(df_week_loop)

    df_week = pd.concat(df_week, axis = 1)

    # X positions
    df_week.index = df_week.index.to_numpy() * 7 + 3  # Mid-point of week
    #df_week.dropna(inplace = True, axis = 1)
    
    # Stats
    #mean_columns = df_week.loc[:, (slice(None), 'mean')] # just the means
    #levene_res = levene(*[mean_columns[col] for col in mean_columns])
    #anova_res = f_oneway(*[mean_columns[col] for col in mean_columns]) # unpack from list comprehension to pass as individual arguments
    #
    #print(f"At {key}\nLevene: {levene_res.statistic:.2f}, p={levene_res.pvalue:.2f} (non-sig. pval = equal variances)\nANOVA F: {anova_res.statistic:.2f}, p={anova_res.pvalue:.2f}\n")

    # Store
    stats_dict[key]["phenology"] = {
        "df_day": df_day,
        "df_week": df_week,
        "y_vars": y_vars,
    #    "levene": levene_res,
    #    "anova": anova_res
    }

In [ ]:
## Get Data
#df = sites["se_st1"].sat_day.copy()
#
# # Z-score
#for var in ["gpp", "gppe", "ndvi"]:
#    df[var] = zscore(df[var], nan_policy = "omit")
#
## Move index to col
#df = df.reset_index().rename(columns={'index': 'date'})
#
## Melt
#df_long = df.melt(
#    id_vars = ["date"], # columns to keep
#    value_vars = ["gppe", "ndvi", "gpp"], # columns to unpivot
#    var_name = "method",
#    value_name = "z_score"  # your standardized phenology value
#)
#
## Unpack Date
#df_long["date"] = pd.to_datetime(df_long["date"])
#df_long["year"] = df_long["date"].dt.year
#df_long["week"] = df_long["date"].dt.isocalendar().week
#
#df_long = df_long.dropna(subset = ["z_score"])
#
#df = df_long
#
#import statsmodels.api as sm
#from statsmodels.formula.api import mixedlm
#
## Convert method to categorical (if not already)
#df['method'] = df['method'].astype('category')
#
## Specify the formula: z-score ~ method, with random intercepts for year and week
## Random intercept for year and week nested within year
#
## Create a combined grouping for week nested in year
#df['year_week'] = df['year'].astype(str) + '_' + df['week'].astype(str)
#
## Fit the linear mixed effect model
#model = mixedlm("z_score ~ method", df, groups=df["year"], 
#                re_formula="1")
#result = model.fit()
#print(result.summary())

## Platform Differences

# Save

In [ ]:
with open("stats.pkl", "wb") as f:
    dill.dump(stats_dict, f)
    
with open("stats_combined.pkl", "wb") as f:
    dill.dump(stats_dict_all, f)